# Assignment 4: Alarm System PYNQ Board #2 
- Run a server and client on the same machine on separate processes
- connect to and disconnect from a remote server
- communicate button presses between PYNQ boards

Part A4.1

Buzzer Module Wiring to PYNQ PMODA
- - pin connected to GND 
- + pin connectted to PMOD PIN2
- Middle pin is not connected

In [12]:
from multiprocessing import Process
from multiprocessing import Event
import threading
import time
from datetime import datetime
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
import socket
import sys
import os
# from server import Server_Main
# from client import Client_Main

btns = base.btns_gpio
stop_program = Event()
button_pressed = True

In [13]:
%%microblaze base.PMODA
#include "gpio.h"
#include "pyprintf.h"
#include <unistd.h>

static int inited = 0;
static gpio pins[8];

void init_pmoda()
{
    if(inited) return;

    for(int i = 0; i < 8; i++)
    { 
        pins[i] = gpio_open(i); 
        gpio_set_direction(pins[i], GPIO_OUT); 
        gpio_write(pins[i], 0); 
    } 

    inited = 1;   
}

void write_gpio(unsigned int pin, unsigned int val)
{ 
    if (!inited) init_pmoda(); 
    if (pin >= 8) return;
    gpio_write(pins[pin], val);
}

void reset_all()
{
    if (!inited) init_pmoda();
    for(int i = 0; i < 8; i++)
        gpio_write(pins[i], 0);
}

// Buzzer
void buzz(unsigned int pin,
          unsigned int freq_hz,
          unsigned int duration_ms)
{
    if (!inited) init_pmoda();

    if (pin >= 8) { pyprintf("pin must be 0-7\n"); return; }
    if (freq_hz == 0) { pyprintf("freq must be > 0\n"); return; }

    // 50% duty cycle
    unsigned int period_us = 1000000u / freq_hz;
    if (period_us == 0) period_us = 1;

    unsigned int half_period = period_us / 2;
    if (half_period == 0) half_period = 1;

    unsigned int cycles = (duration_ms * 1000u) / period_us;

    for (unsigned int i = 0; i < cycles; i++)
    {
        // write value of 1
        gpio_write(pins[pin], 1);
        
        //sleep for 1/(2*tone_freq)
        usleep((useconds_t)half_period);

        // write value of 0
        gpio_write(pins[pin], 0);
        
        //sleep for 1/(2*tone_freq)
        usleep((useconds_t)half_period);
    }

    gpio_write(pins[pin], 0);  // ensure off
}

void beep(unsigned int pin)
{
    // 2 kHz tone for 500 ms
    buzz(pin, 2000, 500);
}

void high_beep(unsigned int pin)
{
    buzz(pin, 3500, 300);
}

In [14]:
# Regular Buzzer
# beep(2)               # PMODA pin 2
buzz(2, 2500, 500)   # 2.5 kHz for 0.5 second

In [15]:
# Loud Buzzer
# beep(2)               # PMODA pin 2
buzz(2, 4500, 500)   # 4.5 kHz for 0.5 second

In [18]:
# High Pitch Buzz
# beep(2)               # PMODA pin 2
buzz(2, 1500, 500)   # 6.5 kHz for 0.5 second

In [19]:
def server():

    HOST = ''
    PORT = 50007

    # sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    print('Socket created')
    
    # 1: Bind the socket to the pynq board <CLIENT-IP> at port <LISTENING-PORT>
    s_server.bind((HOST, PORT))
    print('Socket bind complete')
    
    s_server.listen(1000)
    print('Listening for Client')
    
    # 2: Accept connections
    print('\nWaiting for message from client...')
    print('OR')
    print('Press button 0 to start client...')
    
    conn, addr = s_server.accept()
    
    with conn:
        print('\nConnected by', addr)

        while True:
        
        #Test Code
        # 3: Receive bytes from the connection
            #data = conn.recv(1024)
            # 4: Print the received message
            #print('Received', data)
            #if not data: break
            #conn.sendall(data)
            
            data = conn.recv(1024)

            if not data:
                break

            # print('Received:', data)

            if data == b'DISCONNECT':
                print("\nClient requested disconnect")
                break
            
            # print("Server Shutting Down")

            elif data == b'BUZZ':
                print("Buzzer PWM command received")
                # Code to control buzzer here
                # Buzzer
                # beep(2)               # PMODA pin 0
                buzz(2, 2500, 500)   # 2.5 kHz for 1 second
                time.sleep(0.3)  # debounce delay

            else:
                conn.sendall(data)
            
def client(button_pressed):
    
    # TODO: 
    # 1: Connect the socket (sock) to the <SERVER-IP> and choosen port <LISTENING-PORT>
    # 2: Send the message "Hello world!\n"
    # 3: Close the socket

    # Enter the other PYNQ board's IP Address
    # HOST = '192.168.XXX.XXX'
    # HOST = '192.168.0.204'
    HOST = '192.168.230.1'
    
    # Loopback to IP Address of my PYNQ Board
    
    # HOST = '127.0.0.1'
    PORT = 50007

    s_client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s_client.connect((HOST, PORT))
    print("Client connected to server")

    try:
        s_client.sendall(b'Hello, world')
        data = s_client.recv(1024)
        print('Received from server:', repr(data))

        print("\nPress Button 3 to disconnect...")
        print("Button 1 to send buzzer PWM command")

        # Wait until Button 1 is pressed
        while True:
            if base.buttons[3].read() == 1:
                print("\nButton 3 Pressed - Disconnecting")
                s_client.sendall(b'DISCONNECT')
                break
                
            # Button 2 for sending buzz command
            if base.buttons[1].read() == 1:
                print("Button 1 Pressed - Sending tone frequency")
                s_client.sendall(b'BUZZ')
                # Buzzer
                # beep(2)               # PMODA pin 0
                # buzz(2, 2500, 500)   # 2.5 kHz for 1 second
                # time.sleep(0.3)  # debounce delay
            
            time.sleep(0.1)

    finally:
        s_client.close()
        print("\nClient socket closed")
        return

In [ ]:
# Server process turns on upon code execution
button_pressed = True

 
try:
    print("starting server")

    # Server Process
    p = Process(target=server, args=())
    p.start()
    time.sleep(1)

#    if button_pressed:
#        while True:
#            button_pressed = False
#            while not button_pressed:
                # If button pressed start client
#                if (base.buttons[0].read() == 1):
#                    print("Button 0 Pressed")
#                    button_pressed = True
#                    print("starting client")
#            # Client Process
#            p1 = Process(target=client, args=(button_pressed,))
#            p1.start()
#            p1.join()
#        time.sleep(0.1)

    #else:
    #    p1 = Process(target=client(button_pressed), args=(button_pressed,))
    #    p1.start()
    #    p1.join()
    # button_pressed = False
    # time.sleep(1)
    # p.join()

    # Button 0 Starts the Client
    while base.buttons[0].read() == 0:
        time.sleep(0.1)

    print("Button 0 Pressed")
    print("starting client")
    
    # Client process start
    p1 = Process(target=client, args=(True,))
    p1.start()

    # Client Finishes
    p1.join()

    # Stop server process
    p.terminate()
    print("Server process terminated")
    p.join()
    print("Cell execution complete")

# Kill process if button doesnt work (For debugging)
except KeyboardInterrupt:
    print('Interrupt')
    sys.exit(0)

starting server
Socket created
Socket bind complete
Listening for Client

Waiting for message from client...
OR
Press button 0 to start client...

Connected by ('192.168.230.1', 35638)
Buzzer PWM command received
Buzzer PWM command received
Buzzer PWM command received
Buzzer PWM command received
Buzzer PWM command received
Buzzer PWM command received
Buzzer PWM command received
Buzzer PWM command received
Buzzer PWM command received
Buzzer PWM command received
Buzzer PWM command received
Buzzer PWM command received
Buzzer PWM command received

Client requested disconnect
